In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv("data/training_data_utf8.csv")

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,id,price_per_m2,transaction_ym,plot_area,plot_share,property_type,cadastral,district,contract_type,land_use,land_use_share,protection_zone,building_ban,building_height,building_floors,building_year,longitude,latitude
0,7671,224.351484,2013-12,4279,1.0,Betriebsobjekt,Kagran,Donaustadt,Kaufvertrag,GB,1.00,No,No,NaN,1.0,1990.0,16.464163,48.259071
1,7672,357.995227,2013-12,838,1.0,Abbruchobjekt,Liesing,Liesing,Kaufvertrag,W,1.00,No,No,10.5,2.0,NaN,16.275103,48.133264
2,7673,636.474908,2013-12,817,1.0,"Ein-, Zweifamilienhaus",Atzgersdorf,Liesing,Kaufvertrag,GB,1.00,No,No,3.5,2.0,NaN,16.296926,48.145348
3,7674,906.095552,2013-12,607,1.0,"Ein-, Zweifamilienhaus",Ottakring,Ottakring,Kaufvertrag,W,1.00,No,No,7.5,2.0,NaN,16.290968,48.217580
4,7675,167.941498,2013-12,11897,1.0,unbebaut,Groß Jedlersdorf I,Floridsdorf,Kaufvertrag,W,0.19,No,No,NaN,NaN,NaN,16.418471,48.288753


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44366 entries, 0 to 44365
Data columns (total 18 columns):
id                 44366 non-null int64
price_per_m2       44366 non-null float64
transaction_ym     44366 non-null object
plot_area          44366 non-null int64
plot_share         44366 non-null float64
property_type      44366 non-null object
cadastral          44366 non-null object
district           44294 non-null object
contract_type      44366 non-null object
land_use           44260 non-null object
land_use_share     44320 non-null float64
protection_zone    44366 non-null object
building_ban       44366 non-null object
building_height    7800 non-null object
building_floors    18422 non-null float64
building_year      14739 non-null float64
longitude          32465 non-null float64
latitude           32465 non-null float64
dtypes: float64(7), int64(2), object(9)
memory usage: 6.1+ MB


In [5]:
#describe numerical values
data.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude
count,44366.000000,44366.000000,4.436600e+04,44366.000000,44320.000000,18422.000000,14739.000000,32465.000000,32465.000000
mean,29853.500000,789.062369,1.880226e+03,0.959631,0.956965,2.063402,1913.283737,16.363182,48.207192
std,12807.505358,2276.246856,1.064498e+04,0.163726,0.144435,1.902943,37.464120,0.074558,0.039031
min,7671.000000,0.161937,1.000000e+00,0.000260,0.000000,0.000000,1800.000000,15.605753,47.850748
25%,18762.250000,181.682084,4.120000e+02,1.000000,1.000000,0.000000,1894.000000,16.310912,48.179708
50%,29853.500000,354.343664,5.810000e+02,1.000000,1.000000,2.000000,1900.000000,16.352831,48.208886
75%,40944.750000,726.728429,1.035000e+03,1.000000,1.000000,4.000000,1940.000000,16.407386,48.230576
max,52036.000000,197628.458498,1.187781e+06,1.000000,1.000000,12.000000,2017.000000,16.572405,48.314311


In [6]:
#describe categorical values
data.describe(include="O")

,transaction_ym,property_type,cadastral,district,contract_type,land_use,protection_zone,building_ban,building_height
count,44366,44366,44366,44294,44366,44260,44366,44366,7800
unique,324,16,88,23,11,56,2,2,97
top,1988-12,unbebaut,Eßling,Donaustadt,Kaufvertrag,WI/GBI,No,No,6.5
freq,296,15345,2270,8476,38853,12099,43306,43857,2763


In [7]:
corrmat = data.corr()

In [8]:
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2       1.000000
building_floors    0.385326
plot_share         0.002082
latitude          -0.011045
plot_area         -0.028618
longitude         -0.038409
land_use_share    -0.050230
building_year     -0.051976
id                -0.133134
Name: price_per_m2, dtype: float64

In [9]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [10]:
latitudeCenter = 48.210033
longitudeCenter = 16.363449

In [11]:
for index, row in data.iterrows():
    data.loc[index, 'distance_to_center'] = haversine(row['longitude'], row['latitude'], longitudeCenter, latitudeCenter)

In [12]:
corrmat = data.corr()
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.385326
plot_share            0.002082
latitude             -0.011045
plot_area            -0.028618
longitude            -0.038409
land_use_share       -0.050230
building_year        -0.051976
id                   -0.133134
distance_to_center   -0.268828
Name: price_per_m2, dtype: float64

In [13]:
data = data[data.distance_to_center < 15]
data = data[data.price_per_m2 < 5000]
data.shape

(31708, 19)

In [14]:
data[['building_floors']] = data[['building_floors']].fillna(data[['building_floors']].mean())

In [15]:
corrmat["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.385326
plot_share            0.002082
latitude             -0.011045
plot_area            -0.028618
longitude            -0.038409
land_use_share       -0.050230
building_year        -0.051976
id                   -0.133134
distance_to_center   -0.268828
Name: price_per_m2, dtype: float64

In [16]:
data_fillna = data.fillna(data.mean())

In [17]:
corrmat_fillna = data_fillna.corr()
corrmat_fillna["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.451683
plot_share           -0.004921
latitude             -0.011959
plot_area            -0.070899
building_year        -0.091530
longitude            -0.120077
land_use_share       -0.132532
id                   -0.269039
distance_to_center   -0.477752
Name: price_per_m2, dtype: float64

In [18]:
data_fillna.describe()

,id,price_per_m2,plot_area,plot_share,land_use_share,building_floors,building_year,longitude,latitude,distance_to_center
count,31708.000000,31708.000000,31708.000000,31708.000000,31708.000000,31708.000000,31708.00000,31708.000000,31708.000000,31708.000000
mean,30119.745395,669.368170,1101.934780,0.962179,0.957303,2.136811,1911.42537,16.363218,48.207237,6.313230
std,12830.194229,751.763499,3378.021244,0.155454,0.144111,1.175471,22.63051,0.075222,0.039365,3.225591
min,7671.000000,0.383400,1.000000,0.001000,0.000000,0.000000,1800.00000,16.190731,48.121595,0.042117
25%,19075.750000,224.129779,417.000000,1.000000,1.000000,2.000000,1910.00000,16.309955,48.179050,3.391614
50%,30211.000000,414.513038,550.000000,1.000000,1.000000,2.136811,1911.42537,16.351973,48.209024,6.442939
75%,41334.500000,793.126241,827.000000,1.000000,1.000000,2.136811,1911.42537,16.409113,48.231179,8.720654
max,52034.000000,4992.887624,143858.000000,1.000000,1.000000,12.000000,2017.00000,16.546959,48.314311,14.409620


In [19]:
date = data_fillna.transaction_ym.apply(pd.to_datetime, format="%Y-%m")
relative_share = data_fillna.plot_area * data_fillna.plot_share

In [20]:
data_fillna["relative_share"] = relative_share
data_fillna["year"]= date.dt.year
data_fillna.shape

(31708, 21)

In [21]:
corrmat_fillna = data_fillna.corr()
corrmat_fillna["price_per_m2"].sort_values(ascending=False)

price_per_m2          1.000000
building_floors       0.451683
year                  0.266118
plot_share           -0.004921
latitude             -0.011959
relative_share       -0.066015
plot_area            -0.070899
building_year        -0.091530
longitude            -0.120077
land_use_share       -0.132532
id                   -0.269039
distance_to_center   -0.477752
Name: price_per_m2, dtype: float64

In [22]:
train_data = data_fillna[["price_per_m2", "building_floors", "year", "distance_to_center", "land_use", "relative_share", "district"]]
train_data_cat = train_data[["land_use", "district"]]
train_data_num = train_data[["year", "distance_to_center", "building_floors", "relative_share"]]

In [23]:
data = train_data.drop("price_per_m2", axis=1)
labels = train_data["price_per_m2"].copy()

In [24]:
#data["contract_type"] = data["contract_type"].astype(str)
data["land_use"] = data["land_use"].astype(str)
data["district"] = data["district"].astype(str)
#data["protection_zone"] = data["protection_zone"].astype(str)

In [25]:
#This is not important
from future_encoders import OrdinalEncoder

train_data_cat["district"] = data["district"]
train_data_cat["land_use"] = data["land_use"]

encoder = OrdinalEncoder()
housing_cat_encoded = encoder.fit_transform(train_data_cat)
encoder.categories_

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


[array(['0', '110', 'BG', 'BG/IG', 'E', 'E div.', 'EKL', 'EKLW', 'EPK',
        'ESP', 'EW', 'Ekl', 'Eklw', 'Epk', 'Epl', 'Esp', 'Friedhof', 'G',
        'GB', 'GBBG', 'GBGV', 'GS', 'GV', 'IG', 'L', 'P', 'SO',
        'SO/sonstiges', 'SPK', 'SWW', 'SWWL', 'Spk', 'Sww', 'SwwL', 'VB',
        'Verkehrsband', 'Vfl', 'W', 'WI/GBI', 'WII/GBII', 'WIII/GBIII',
        'WIV/GBIV', 'WV/GBV', 'WVI/GBVI', 'Wklw', 'gv', 'nan', 'w', 'we'],
       dtype=object),
 array(['Alsergrund', 'Brigittenau', 'Donaustadt', 'Döbling', 'Favoriten',
        'Floridsdorf', 'Fünfhaus', 'Hernals', 'Hietzing', 'Innere Stadt',
        'Josefstadt', 'Landstraße', 'Leopoldstadt', 'Liesing',
        'Margareten', 'Mariahilf', 'Meidling', 'Neubau', 'Ottakring',
        'Penzing', 'Simmering', 'Wieden', 'Währing', 'nan'], dtype=object)]

In [26]:
#This is not important
data_fillna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31708 entries, 0 to 44363
Data columns (total 21 columns):
id                    31708 non-null int64
price_per_m2          31708 non-null float64
transaction_ym        31708 non-null object
plot_area             31708 non-null int64
plot_share            31708 non-null float64
property_type         31708 non-null object
cadastral             31708 non-null object
district              31651 non-null object
contract_type         31708 non-null object
land_use              31639 non-null object
land_use_share        31708 non-null float64
protection_zone       31708 non-null object
building_ban          31708 non-null object
building_height       6160 non-null object
building_floors       31708 non-null float64
building_year         31708 non-null float64
longitude             31708 non-null float64
latitude              31708 non-null float64
distance_to_center    31708 non-null float64
relative_share        31708 non-null float64
year 

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [28]:
from future_encoders import ColumnTransformer
from future_encoders import OneHotEncoder

num_attribs = list(train_data_num)
cat_attribs = list(train_data_cat)

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [29]:
num_attribs

['year', 'distance_to_center', 'building_floors', 'relative_share']

In [30]:
cat_attribs

['land_use', 'district']

In [31]:
data_prepared = full_pipeline.fit_transform(data)
data_prepared

<31708x77 sparse matrix of type '<class 'numpy.float64'>'
	with 172669 stored elements in Compressed Sparse Row format>

In [32]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
lin_reg.fit(data_prepared, labels)

scores = cross_val_score(lin_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
lin_reg_rmse_scores = np.sqrt(-scores)

display_scores(lin_reg_rmse_scores)

Scores: [808.12770788 636.41497597 576.69471536 549.26574999 507.19696686
 551.5353371  495.35952868 482.1640242  445.04438254 463.72562957]
Mean: 551.552901815762
Standard Deviation 101.32482411231531


In [34]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(data_prepared, labels)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [35]:
scores = cross_val_score(ridge_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores = np.sqrt(-scores)

display_scores(ridge_reg_rmse_scores)

Scores: [808.61933141 636.32559143 576.40478797 548.7037131  507.3480241
 552.26835787 495.24215906 482.05538908 444.9934802  463.9431836 ]
Mean: 551.5904017805569
Standard Deviation 101.43029052877199


In [36]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(data_prepared, labels)

scores = cross_val_score(tree_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
tree_reg_rmse_scores = np.sqrt(-scores)

display_scores(tree_reg_rmse_scores)

Scores: [832.87211928 798.53399915 713.03792883 702.33333872 608.61957286
 625.96861582 601.91796267 628.75783368 583.06896092 568.29530866]
Mean: 666.3405640582002
Standard Deviation 87.03755458808584


In [37]:
feature_importances = tree_reg.feature_importances_
attributes = num_attribs + cat_attribs + list(encoder.categories_[1]) +  list(encoder.categories_[0])
sorted(zip(feature_importances, attributes), reverse=True)

[(0.29240837210382065, 'building_floors'),
 (0.2597969305022091, 'distance_to_center'),
 (0.17314481264698264, 'year'),
 (0.14032120865602019, 'relative_share'),
 (0.0124306157923797, 'Spk'),
 (0.009588133411638841, 'Sww'),
 (0.008500349770090355, 'Friedhof'),
 (0.00744168161057282, 'Ekl'),
 (0.0067162690511957815, 'Verkehrsband'),
 (0.005580151587569536, 'SO'),
 (0.005125914930841361, 'L'),
 (0.004542201289527256, 'VB'),
 (0.00451477814365558, 'Wklw'),
 (0.004439122938482597, 'Esp'),
 (0.004346732218995108, 'WIV/GBIV'),
 (0.003984623318849529, 'gv'),
 (0.003957616621830375, 'IG'),
 (0.0039005050760676624, 'WIII/GBIII'),
 (0.003832764871299201, 'Epl'),
 (0.0037530655376999786, 'W'),
 (0.003729016859042963, 'Epk'),
 (0.0035110471823528305, 'WI/GBI'),
 (0.0031682167041970563, 'SWWL'),
 (0.0031438385380736793, 'SWW'),
 (0.003116575729883485, 'SO/sonstiges'),
 (0.0029351577511383463, 'Meidling'),
 (0.002899436025317554, 'WII/GBII'),
 (0.0025774410554562175, 'Eklw'),
 (0.0025256765303794753